In [1]:
%cd ..

import numpy as np
import pandas as pd
from src.convolution import *
from src.linear import *
from src.loss import *
from src.activation import *
from src.encapsulation import *
from utils.mltools import *


def load_usps(fn):
    with open(fn, "r") as f:
        f.readline()
        data = [[float(X) for X in l.split()] for l in f if len(l.split()) > 2]
    tmp = np.array(data)
    return tmp[:, 1:], tmp[:, 0].astype(int)


def get_usps(l, X, y):
    if type(l) != list:
        resx = X[y == l, :]
        resy = y[y == l]
        return resx, resy
    tmp = list(zip(*[get_usps(i, X, y) for i in l]))
    tmpx, tmpy = np.vstack(tmp[0]), np.hstack(tmp[1])
    return tmpx, tmpy


alltrainx, alltrainy = load_usps("data/USPS_train.txt")
alltestx, alltesty = load_usps("data/USPS_test.txt")


def load_one_class(number):
    X_train, y_train = get_usps(number, alltrainx, alltrainy)
    X_test, y_test = get_usps(number, alltestx, alltesty)
    y_train = np.where(y_train == number, -1, 1).reshape(-1, 1)
    y_test = np.where(y_test == number, -1, 1).reshape(-1, 1)
    return X_train, y_train, X_test, y_test


def load_two_classes(neg, pos):
    X_train, y_train = get_usps([neg, pos], alltrainx, alltrainy)
    X_test, y_test = get_usps([neg, pos], alltestx, alltesty)
    y_train = np.where(y_train == neg, -1, 1).reshape(-1, 1)
    y_test = np.where(y_test == neg, -1, 1).reshape(-1, 1)
    return X_train, y_train, X_test, y_test


d:\~Perso\Etudes\DAC\NeuralNetworksDIY


In [2]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler

alltrainy = OneHotEncoder().fit_transform(alltrainy.reshape(-1, 1)).toarray()
alltrainy.shape

(7291, 10)

In [7]:
net = Sequential(
    Conv1D(3, 1, 32, 1, init_type="xavier_normal"),
    MaxPool1D(2, 2),
    Flatten(),
    Linear(4064, 100, init_type="xavier_normal"),
    ReLU(),
    Linear(100, 10, init_type="xavier_normal"),
)
optimizer = Optim(net, CrossEntropyLoss(), eps=0.001)
result_df = optimizer.SGD_eval(alltrainx[:,:,np.newaxis], alltrainy, 1000, 25, test_size=0.33, return_dataframe=True)


100%|██████████| 25/25 [09:04<00:00, 21.77s/it]


In [8]:
result_df

,epoch,loss_test,loss_train,score_train,score_test
0,0,0.297328,0.187851,0.941032,0.919817
1,1,0.174563,0.156314,0.959050,0.939759
2,2,0.144112,0.142294,0.968468,0.946406
3,3,0.127571,0.137057,0.976249,0.950976
4,4,0.116140,0.130009,0.979934,0.955962
5,5,0.107822,0.124893,0.980753,0.960532
6,6,0.101160,0.122228,0.983415,0.962194
7,7,0.095764,0.118208,0.985463,0.962609
8,8,0.091532,0.115878,0.986896,0.963025
9,9,0.087558,0.117560,0.987510,0.965517


In [ ]:
net = Sequential(
    Conv1D(3, 1, 32, 1, init_type="xavier_normal"),
    ReLU(),
    MaxPool1D(2, 2),
    Conv1D(3, 32, 32),
    ReLU(),
    MaxPool1D(2, 2),
    Flatten(),
    Linear(1984, 10),
)
optimizer = Optim(net, CrossEntropyLoss(), eps=0.001)
lossList = optimizer.SGD(alltrainx[:,:,np.newaxis], alltrainy, 1000, 10)
print(lossList)
pd.Series(lossList).plot()


In [2]:
import os
import pandas as pd
from skimage import io, transform
from PIL import Image


def load_data(root_dir):
    size = (540, 420)
    X = []
    for filepath in os.listdir(root_dir + "/X"):
        imgpath = os.path.join(root_dir, "X", filepath)
        image = io.imread(imgpath)
        image = transform.resize(image, size)
        X.append(image)
    X = np.array(X)

    y = []
    for filepath in os.listdir(root_dir + "/y"):
        imgpath = os.path.join(root_dir, "y", filepath)
        image = io.imread(imgpath)
        image = transform.resize(image, size)
        y.append(image)
    y = np.array(y)

    return X, y


X, y = load_data("./data/dirty_documents")


In [3]:
X[:, np.newaxis, :, :].shape


(144, 1, 540, 420)

In [4]:
X[:, :, :, np.newaxis].shape


(144, 540, 420, 1)